<a href="https://colab.research.google.com/github/NimaZah/DoubleML/blob/main/DoubleML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install doubleml==0.3.0
!pip install sklearn
!pip install statsmodels==0.10.1
import math
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import NMF
from sklearn.exceptions import ConvergenceWarning
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Lasso, LassoCV
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from warnings import simplefilter
simplefilter("ignore", category=ConvergenceWarning)

In [ ]:
# fetch teh data, then create a new document
# by taking the first 1000 words of each document. This is done to reduce the
# size of the documents, and to make the documents more manageable.

# Remove documents that have no words after filtering. Remove documents that have
# less than 25 words. Next, create a vocabulary of all the words that appear
# in the documents.

# Then create a tf-idf vector for each document. A tf-idf vector is a vector
# that represents the importance of each word in the document. The importance
# increases proportionally to the number of times a word 
# appears in the document, but is offset by the frequency of the word in the 
# corpus, which helps to adjust for the fact that some words appear more
# frequently in general.

newsgroup_data = fetch_20newsgroups(
    subset="train", remove=["headers", "footers", "quotes"], shuffle=False
)

target_name_to_label = {
    "comp.graphics": 1,
    "comp.os.ms-windows.misc": 1,
    "comp.sys.ibm.pc.hardware": 1,
    "comp.sys.mac.hardware": 1,
    "comp.windows.x": 1,
    "misc.forsale": 2,
    "rec.autos": 3,
    "rec.motorcycles": 3,
    "rec.sport.baseball": 4,
    "rec.sport.hockey": 4,
    "sci.crypt": 5,
    "sci.electronics": 6,
    "sci.med": 7,
    "sci.space": 8,
    "alt.atheism": 9,
    "soc.religion.christian": 9,
    "talk.religion.misc": 9,
    "talk.politics.guns": 10,
    "talk.politics.mideast": 10,
    "talk.politics.misc": 10,
}
label_names = [
    "comp",
    "sale",
    "auto",
    "sport",
    "crypt",
    "electronics",
    "med",
    "space",
    "religion",
    "politics",
]
labels = [
    target_name_to_label[newsgroup_data.target_names[original_label]]
    for original_label in newsgroup_data.target
]
labels = np.array(labels)

vectorizer = TfidfVectorizer(min_df=25, max_df=0.01)
tfidf_vectors = vectorizer.fit_transform(newsgroup_data.data).toarray()
print(tfidf_vectors.shape, labels.shape)

documents = []
dropped_document_idx = set([])
vocabset = set(vectorizer.get_feature_names())
for idx, text in enumerate(newsgroup_data.data):
    document = []
    for token in text.split(" "):
        if token not in vocabset:
            continue
        document.append(token)

    if len(document) == 0:
        dropped_document_idx.add(idx)
        continue

    document = " ".join(document)
    if len(document) > 1000:
        document = document[:1000]

    documents.append(document)

tfidf_vectors = np.array(
    [
        tfidf_vectors[idx, :]
        for idx in range(tfidf_vectors.shape[0])
        if idx not in dropped_document_idx
    ]
)
labels = np.array(
    [labels[idx] for idx in range(len(labels)) if idx not in dropped_document_idx]
)
print(tfidf_vectors.shape, labels.shape, len(documents))

(11314, 3882) (11314,)
(9816, 3882) (9816,) 9816


In [ ]:
# create a simulated dataset where the treatment effect is non-zero and there is
# an unobserved confounder. The confounder is a binary variable that is generated
# randomly and depends on the label of the text. The treatment effect is
# generated randomly and depends on the confounder. The outcome is generated
# randomly and depends on the treatment effect and confounder.

def sigmoid(x):
    return 1. / (1. + math.exp(-x))
simulated_data = []
for idx in range(tfidf_vectors.shape[0]):
    confounder = np.random.binomial(1, sigmoid(labels[idx]))
    treatment_effect = np.random.binomial(1, sigmoid(confounder))
    outcome = np.random.binomial(1, sigmoid(treatment_effect + confounder))
    simulated_data.append([confounder, treatment_effect, outcome])
simulated_data = np.array(simulated_data)


def sigmoid(x):
    return 1. / (1. + math.exp(-x))

simulated_data = []
for idx in tqdm(range(tfidf_vectors.shape[0])):    
    # confounder
    U = int(labels[idx]==9) 
        
    # treatment
    Z = 2.*U + np.random.normal(loc=0.5, scale=5.0)  
    
    # binary
    Y = -0.5 + 0.05*Z + 5.0*U + np.random.normal(0.0, 0.1)
    
    simulated_data.append([Y, Z, U] + list(tfidf_vectors[idx]))

simulated_data = np.array(simulated_data)

  0%|          | 0/9816 [00:00<?, ?it/s]

In [ ]:
# df = pd.DataFrame(simulated_data)
# df.columns = ["Y", "Z", "U"] + vectorizer.get_feature_names()
# df.head()
df = pd.DataFrame(simulated_data)
df.columns = ["Y", "Z", "U"] + ["Word" + str(i) for i in range(tfidf_vectors.shape[1])]
df

,Y,Z,U,Word0,Word1,Word2,Word3,Word4,Word5,Word6,...,Word3872,Word3873,Word3874,Word3875,Word3876,Word3877,Word3878,Word3879,Word3880,Word3881
0,-0.604140,-0.235388,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,-0.348622,1.078087,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.148449,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,-0.492682,3.149868,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.021589,0.0
3,-0.406508,0.132299,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,4.910360,10.900698,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9811,-0.801077,-2.860675,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
9812,-0.318373,1.002237,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
9813,-0.315978,1.365846,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
9814,0.079044,8.984593,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [ ]:
%%time
y = simulated_data[:, 0]
X = sm.add_constant(simulated_data[:, 1:2])
model = sm.OLS(endog=y, exog=X)
res = model.fit(method="pinv", maxiter=100)
print(res.summary(yname="Y", xname=["const", "Z"]))

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.084
Model:                            OLS   Adj. R-squared:                  0.084
Method:                 Least Squares   F-statistic:                     895.4
Date:                Thu, 14 Apr 2022   Prob (F-statistic):          2.39e-188
Time:                        18:09:31   Log-Likelihood:                -19020.
No. Observations:                9816   AIC:                         3.804e+04
Df Residuals:                    9814   BIC:                         3.806e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1210      0.017      7.040      0.0

In [ ]:
# Treatment Effect analysisi: 2
%%time
y = simulated_data[:, 0]
X = sm.add_constant(simulated_data[:, 1:3])
model = sm.OLS(endog=y, exog=X)
res = model.fit(method="pinv", maxiter=2000)
print(res.summary(yname="Y", xname=["const", "Z", "U"]))

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.997
Model:                            OLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                 1.506e+06
Date:                Thu, 14 Apr 2022   Prob (F-statistic):               0.00
Time:                        18:11:45   Log-Likelihood:                 8673.3
No. Observations:                9816   AIC:                        -1.734e+04
Df Residuals:                    9813   BIC:                        -1.732e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4999      0.001   -458.897      0.0

### Playing around with the number of topics (two approaches)

In [ ]:

def get_nmf_topics(model, n_top_words):
    #the word ids obtained need to be reverse-mapped to the words so 
    # we can print the topic names.
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(10):
        
        #for each topic, we have words ids, weights, and topics.
        words_ids = model.components_[i].argsort()[:-20 - 1:-1]
        words = [feat_names[key] for key in words_ids]
        weights = model.components_[i][words_ids]
        
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);

nmf_model = NMF(n_components=10, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf_vectors)
get_nmf_topics(nmf_model, 10)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10
0,geb,nhl,armenian,ide,motif,nsa,objective,oil,msg,vga
1,cadre,pittsburgh,armenians,hd,widget,escrow,morality,valve,chinese,256
2,dsl,detroit,turkish,motherboard,xt,des,subjective,energy,foods,svga
3,chastity,rangers,turkey,isa,x11r5,privacy,absolute,bolt,eat,colors
4,n3jxp,cup,armenia,bios,widgets,crypto,immoral,tube,taste,monitors
5,pitt,fans,turks,meg,toolkit,enforcement,observations,stick,reaction,ati
6,intellect,montreal,genocide,simms,interviews,encrypted,relative,bmw,effects,vesa
7,shameful,playoffs,russian,vlb,mailing,agencies,goals,hole,brain,modes
8,skepticism,leafs,azerbaijan,transfer,pixmap,rsa,species,engines,salt,lc
9,surrender,islanders,soviet,cache,resource,proposal,animals,dealer,causes,meg


In [ ]:
# takes the tfidf_vectors and transforms them into the topic vector. 
# The topic vector is a vector of numbers that represent the topic of the document. 
# The nmf.transform() method returns a matrix of topic vectors. 

%%time
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

numtopics = 50
nmf = NMF(n_components=numtopics).fit(tfidf_vectors)
tfidf_feature_names = vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, 10)
nmf_topic_weights = nmf.transform(tfidf_vectors)

Topic #0: geb cadre dsl n3jxp chastity pitt intellect surrender shameful skepticism
Topic #1: criminals firearms auto criminal weapon semi violent firearm crimes handgun
Topic #2: armenian armenians turkish armenia genocide russian soviet azerbaijan turkey turks
Topic #3: cpu fpu mhz upgrade cache nubus slot processor iisi pds
Topic #4: motif xt toolkit mailing x11r5 interviews linux widgets fixing icon
Topic #5: nhl pittsburgh rangers detroit cup montreal playoffs leafs espn islanders
Topic #6: sin heaven spirit scripture mary eternal holy catholic matthew kingdom
Topic #7: ide hd isa transfer sec burst mfm dma slave master
Topic #8: msg chinese foods eat reaction taste brain effects salt sick
Topic #9: font fonts characters postscript character ps laser printers zip converted
Topic #10: moon launch orbit lunar shuttle mission satellite spacecraft solar mars
Topic #11: objective morality subjective absolute animals goals relative observations immoral species
Topic #12: ride rear ridin

### Using Double ML to estimate the treatment effect
$$
Y = \beta_0 + \beta_1 Z + \beta_2 Word0 + \beta_3 Word1 + \beta_4 Word2 + \beta_5 Word3 + \epsilon
$$

In [ ]:
# create a data object, which is a Pandas dataframe with a column called "Y" 
# that contains the outcome variable, a column called "Z" that contains the
# treatment variable, and a column called "Word0" that contains the first 
# predictor variable. The other columns are called "Word1", "Word2", etc.

# The data object is then passed to the DoubleMLPLR constructor, which takes the
# data object and two predictor objects as arguments. The DoubleMLPLR object
# then fits the treatment and outcome predictors to the data.

# The DoubleMLPLR object has a method called "summary" that prints out the
# coefficients of the treatment and outcome predictors.

%%time
data = doubleml.DoubleMLData(data=df, y_col="Y", d_cols="Z",
                             x_cols=["Word" + str(i) for i in range(len(df.columns)-3)])

treatment_predictor = LassoCV(cv=2, n_alphas=1, verbose=0, n_jobs=-1)
outcome_predictor = LassoCV(cv=2, n_alphas=1, verbose=0, n_jobs=-1)

estimator = doubleml.DoubleMLPLR(data, treatment_predictor, outcome_predictor,
                                 apply_cross_fitting=False, n_folds=2)
estimator.fit()
print(estimator.summary)

       coef   std err         t         P>|t|     2.5 %    97.5 %
Z  0.042494  0.002642  16.08506  3.247363e-58  0.037317  0.047672
CPU times: user 43 s, sys: 4.72 s, total: 47.7 s
Wall time: 1min 26s
